In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
import tkinter
from tkinter import ttk
import numpy as np      
from PIL import Image, ImageTk

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angles(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle
    
    return angle

In [3]:
def img(cap, pose):
    ret,frame = cap.read()
    

    #recolor bgr to rgb
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    #making detection
    results = pose.process(image)

    #recolor rgb to bgr
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

    #Extract landmarks
    try: 
        landmarks = results.pose_landmarks.landmark
        elbow = list()
        shoulder = list()
        wrist = list()
        hip = list()
        knee = list()
        ankle = list()
        shoulder.append([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
        elbow.append([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
        wrist.append([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] )
        shoulder.append([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y])
        elbow.append([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])
        wrist.append([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
        hip.append([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        hip.append([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        knee.append([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
        knee.append([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y])
        ankle.append([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
        ankle.append([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
        #calculate angle
        angle = list()
        angle.append(calculate_angles(elbow[0],shoulder[0],hip[0]))
        angle.append(calculate_angles(elbow[1],shoulder[1],hip[1]))
        angle.append(calculate_angles(shoulder[0],elbow[0],wrist[0]))
        angle.append(calculate_angles(shoulder[1],elbow[1],wrist[1]))
        angle.append(calculate_angles(shoulder[0],hip[0],knee[0]))
        angle.append(calculate_angles(shoulder[1],hip[1],knee[1]))
        angle.append(calculate_angles(hip[0],knee[0],ankle[0]))
        angle.append(calculate_angles(hip[1],knee[1],ankle[1]))
        #visualize
        if(angle[0]>170 and angle[1]>170 and (angle[6] < 80 or angle[7] < 80)):
          msg = "tree pose"
        elif(angle[0] >170 or angle[1]>170 and (angle[6] <130 and angle[7] > 140) or (angle[7] <130 and angle[6] > 140)):
          msg = "warrior pose"
        elif(angle[0] >=90 and angle[1]>=90 and angle[4] >170 and angle[5] > 170):
          msg = "plank pose"
        elif(angle[2] >170 and angle[3]>170 and angle[4] <95 and angle[5] <95 ):
          msg = "staff pose"
        else:
          msg = "standing pose"
        cv2.putText(image, msg, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    except:
        pass
  
    #reder detection
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(57, 255, 20), thickness=5, circle_radius= 3),
                              mp_drawing.DrawingSpec(color=(57, 255, 20), thickness = 5, circle_radius= 3))
    return image

In [4]:
def win(cv_image):
    
    frame=cv2.cvtColor(cv_image,cv2.COLOR_BGR2RGB)
    img_update = ImageTk.PhotoImage(Image.fromarray(frame))
    show_image.configure(image=img_update)
    show_image.image=img_update
    # show_image.after(1000, win())
    

In [5]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:  
  while cap.isOpened():
    ret, frame = cap.read()
    #get detections
    cv_image = img(cap,pose)
    #gui
    # win(cv_image)
    window=tkinter.Tk()
    window.title("Example about handy CV2 and tkinter combination...")

    # frame=cv2.cvtColor(cv_image,cv2.COLOR_BGR2RGB)
    # img = ImageTk.PhotoImage(Image.fromarray(frame))

   

    message="check your yoga poses here"
    show_text=tkinter.Label(window,text=message)
    show_text.grid(row=1,column=1,pady=1,padx=10)
    
  #Update the image to tkinter...
    
    show_image=tkinter.Label(window)
    show_image.grid(row=0,column=0,columnspan=3,pady=1,padx=10)
    # frame=cv2.cvtColor(cv_image,cv2.COLOR_BGR2RGB)
    # img_update = ImageTk.PhotoImage(Image.fromarray(frame))
    # show_image.configure(image=img_update)
    # show_image.image=img_update
    # window.bind("<Reutrn>",)
    # window.after(1000,win(cv_image))
    # cv2.imshow('yoga',cv_image)
    
    # window.mainloop()
    cv2.imshow('yoga',frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()